In [1]:
DATA_NAME = 'syntheticMRI2D-axial' 
TRANSFORM = 'wavelet-horizontal'
CHANNEL = ''

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_horizontal_wavelet_axial_syntheticMRI2D.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,1134723.81,7.85,-73.89,20.54,-8.21,-4.60,-0.24,0.22
3,7.85,5155075.91,-168.76,-1.13,31.84,16.16,1.30,-0.10
4,-73.89,-168.76,2791235.24,129.76,3.63,-5.08,-1.14,0.31
5,20.54,-1.13,129.76,544185.38,-22.57,-0.94,0.66,-0.04
6,-8.21,31.84,3.63,-22.57,188879.21,-2.48,0.25,0.03
7,-4.60,16.16,-5.08,-0.94,-2.48,27441.74,0.08,0.00
8,-0.24,1.30,-1.14,0.66,0.25,0.08,1838.11,-0.01
9,0.22,-0.10,0.31,-0.04,0.03,0.00,-0.01,78.43


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9
2,1.00000,0.00000,-0.00004,0.00003,-0.00002,-0.00003,-0.00001,0.00002
3,0.00000,1.00000,-0.00004,-0.00000,0.00003,0.00004,0.00001,-0.00001
4,-0.00004,-0.00004,1.00000,0.00011,0.00000,-0.00002,-0.00002,0.00002
5,0.00003,-0.00000,0.00011,1.00000,-0.00007,-0.00001,0.00002,-0.00001
6,-0.00002,0.00003,0.00000,-0.00007,1.00000,-0.00003,0.00001,0.00001
7,-0.00003,0.00004,-0.00002,-0.00001,-0.00003,1.00000,0.00001,0.00000
8,-0.00001,0.00001,-0.00002,0.00002,0.00001,0.00001,1.00000,-0.00003
9,0.00002,-0.00001,0.00002,-0.00001,0.00001,0.00000,-0.00003,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

326.1452748375914

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[3.35342724e+12 9.52970466e+11 1.54313778e+11 3.37035856e+10
 3.79142771e+09 7.16023859e+07 2.41498351e+05 2.57764283e-28]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,-0.030568,0.993735,-0.106472,-0.014041,-0.004810,-0.000695,-0.000047,-0.000002
1,-0.083834,0.103012,0.990496,-0.033873,-0.011311,-0.001636,-0.000110,-0.000005
2,0.988046,0.036515,0.075164,-0.125278,-0.032656,-0.004594,-0.000308,-0.000013
3,0.117768,0.021370,0.040285,0.986544,-0.103045,-0.012740,-0.000850,-0.000036
4,0.043214,0.009318,0.017257,0.096974,0.992834,-0.051116,-0.003325,-0.000142
5,0.008079,0.001773,0.003283,0.016862,0.049198,0.998033,-0.033812,-0.001433
6,0.000809,0.000178,0.000329,0.001684,0.004855,0.033491,0.998512,-0.042709
7,0.000069,0.000015,0.000028,0.000144,0.000415,0.002855,0.042635,0.999087


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[1.0305682807169079,
 0.896988093871751,
 0.9248357714531185,
 0.013455628487099358,
 0.007165652749126394,
 0.0019665469857432782,
 0.0014875652731809463,
 0.000913474717883167]